###### Assignment 1 Submission
**Report & Code**

Name: Zenas Huang

(a) Write a program to construct a decision tree pased on the idea of splitting by Information Gain.

Description: 

The program primarily uses pandas dataframes, arrays, and dictionaries data structures. Recursion is used to build up the decision tree as a nested dictionary beginning with the feature that has the maximum information gain. Helper functions are written at the beginning to compute the entropy and information gain of any subset of the data. The buildTree function then uses these to compute the information gain of each feature and selects the features corresponding to maximum information gain to partition on. Then this feature is removed from the feature space and a subtree is built under that feature represented by a nested dictionary containing the feature's possible values, with the just partitioned feature as its key. The dataframe is then sliced into subsets according to the value that the partition feature takes on and each subset builds its own subtree by recursive call to buildTree, which proceeds until the subset is pure or the list of features to partition on has been exhausted.

In [1]:
import numpy as np
import pandas as pd

def entropy(labelCol):
    #compute the fraction of each class and the entropy
    values, freq = np.unique(labelCol,return_counts = True)
    entropy = 0
    for i in range(len(values)):
        pi = freq[i]/np.sum(freq)
        entropy += -pi*np.log2(pi)    
    return entropy  

def infoGain(data,partitionFeature):  
    #Calculate the entropy of the data before  
    initialEntropy = entropy(data['Enjoy'])  
    #find the values and frequencies to compute the pi proportions for entropy computation    
    values, freq = np.unique(data[partitionFeature],return_counts=True)  

    postEntropy = 0 
    for i in range(len(values)):
        pi = freq[i]/np.sum(freq) #group weight
        subG = data.where(data[partitionFeature]==freq[i]).dropna()['Enjoy'] #group
        subEnt = entropy(subG) #group entropy
        postEntropy += pi*subEnt

    #Calculate the information gain per lecture formula I(Before) - I(After)
    InformationGain = initialEntropy - postEntropy  
    return InformationGain 

def buildTree(data,featuresList):
    #set a default Classlabel value using the majority class in the subset
    parentClassLabel = np.unique(data['Enjoy'])[np.argmax(np.unique(data['Enjoy'],return_counts=True)[1])]  

    #stopping cases
    #If pure data then return the label value  
    if len(np.unique(data['Enjoy'])) == 1:  
        return np.unique(data['Enjoy'])[0]
    #If no more features to split on, return class of the immediate parent  
    elif len(featuresList) == 0:
        return parentClassLabel 

    #build the decision tree
    else:
        #Find and store the Infogains of each feature in the feature space and store as list
        featureIGs = [infoGain(data,feature) for feature in featuresList]
        #Get the feature which gives most information gain for the split
        partitionFeature = featuresList[np.argmax(featureIGs)]  
        #Create tree as a nested dictionary. where each root is the next partition feature    
        tree = {partitionFeature:{}}  

        #Remove the feature that was just partitioned from the feature  
        featuresList = [f for f in featuresList if f != partitionFeature]  

        #Make a subtree for each value of the feature  
        for featureValue in np.unique(data[partitionFeature]):  
            featureValue = featureValue
            #partition dataset into subsets for each value of the partitionfeature  
            subset = data.where(data[partitionFeature] == featureValue).dropna()    
            #Recursively build tree on the subset with the new parameters  
            subtree = buildTree(subset,featuresList)
            #Add the sub tree, grown from the sub_dataset to the tree under the root node  
            tree[partitionFeature][featureValue] = subtree
        return(tree)

def predict(instance,tree,data):  
    #set a default prediction as the majority class label if the instance's path
    #doesnt exist in the tree
    defaultPred = np.unique(data['Enjoy'])[np.argmax(np.unique(data['Enjoy'],return_counts=True)[1])]

    #traverse the tree by checking the instance feature values against the tree 
    #using multidim indexing of the nested dictionary tree
    for key in list(instance.keys()):  
        if key in list(tree.keys()):  
            try:  
                prediction = tree[key][instance[key]]   
            except:  
                return defaultPred

            prediction = tree[key][instance[key]]  
            #if the result is a subtree then use recursion to continue 
            #traversing the tree until its not (leaf reached)
            if type(prediction) is dict:
                return predict(instance,prediction,data)
            else:  
                return prediction

(b) Run the program on the datafile

In [2]:
#Specify location of data
filepath = "./dt_data.txt"
#Read in the provided data as Dataframe
data = pd.read_csv(filepath,skipinitialspace=True,sep=':',header=0)
data = data['(Occupied, Price, Music, Location, VIP, Favorite Beer, Enjoy)'].str.split(',',expand=True)
data.columns = ['Occupied', 'Price', 'Music', 'Location', 'VIP', 'Favorite Beer', 'Enjoy']
#Strip whitespaces from the data
for key in data.keys():
    data[key] = data[key].str.strip()

In [3]:
#Visualize the dataframe to ensure it was readin properly
data.head()

,Occupied,Price,Music,Location,VIP,Favorite Beer,Enjoy
1,High,Expensive,Loud,Talpiot,No,No,No;
2,High,Expensive,Loud,City-Center,Yes,No,Yes;
3,Moderate,Normal,Quiet,City-Center,No,Yes,Yes;
4,Moderate,Expensive,Quiet,German-Colony,No,No,No;
5,Moderate,Expensive,Quiet,German-Colony,Yes,Yes,Yes;


In [4]:
#Run the program
tree = buildTree(data,data.columns[:-1])

In [5]:
#Print the Decision Tree as a nested dictionary
tree

{'Occupied': {'High': {'Price': {'Cheap': 'Yes;',
    'Expensive': {'Music': {'Loud': {'Location': {'City-Center': 'Yes;',
        'Talpiot': 'No;'}}}},
    'Normal': {'Music': {'Loud': 'Yes;', 'Quiet': 'No;'}}}},
  'Low': {'Price': {'Cheap': {'Music': {'Loud': 'Yes;', 'Quiet': 'No;'}},
    'Expensive': 'No;',
    'Normal': {'Music': {'Quiet': {'Location': {'City-Center': {'VIP': {'No': {'Favorite Beer': {'No': 'No;'}}}},
        'Ein-Karem': 'No;'}}}}}},
  'Moderate': {'Price': {'Cheap': {'Music': {'Loud': {'Location': {'Mahane-Yehuda': 'Yes;',
        'Talpiot': 'No;'}}}},
    'Expensive': {'Music': {'Quiet': {'Location': {'German-Colony': {'VIP': {'No': 'No;',
          'Yes': 'Yes;'}},
        'Mahane-Yehuda': 'Yes;'}}}},
    'Normal': 'Yes;'}}}}

Description:

In the Nested Dictionary representation of the Decision Tree, each Key represents a split attribute. For example the outermost Key is 'Occupied' meaning that this feature had the highest information gain in the first partition. 'Occupied' takes on values of 'High', 'Moderate', and 'Low',  each of which is a key corresponding to a dictionary at which the next best feature question is asked, which is its own subtree represented by a nested dictionary. At the terminating leafs of the tree (which correspond to the innermost nested dictionary), the values are not nested dictionaries but simple strings of 'Yes;' or 'No;' which correspond to the label class of whether the night out was enjoyed when following along that path of feature questions.

In [6]:
#Verify that the tree predicts correctly using a known instance in the training set
# row 1 (row index 0), should be 'No;'
known_pred = data.iloc[0,:-1]
known_pred = known_pred.to_dict()
print(known_pred)
print('The prediction is: ', predict(known_pred,tree,data))

{'Occupied': 'High', 'Price': 'Expensive', 'Music': 'Loud', 'Location': 'Talpiot', 'VIP': 'No', 'Favorite Beer': 'No'}
The prediction is:  No;


(c) Make a prediction for (occupied = Moderate; price = Cheap; music = Loud; location = City-Center; VIP=No; Favorite Beer = No)

In [7]:
#Create the new instance as a dictionary
new_instance = {'Occupied':'Moderate','Price':' Cheap','Music':'Loud',\
                'Location':'City-Center','VIP':'No','Favorite Beer':'No'}
#Predict Enjoyment on the New Instance
predict(new_instance, tree, data)

'Yes;'

The Decision Tree predicts that **Yes**, this night-out in Jersualem for the coming New Year's Eve ought to be enjoyed.